In [2]:
# Genetic Programming
# Author: https://github.com/lowerkey/genetic_programming
# Task Source: http://burakkanber.com/blog/machine-learning-genetic-algorithms-in-javascript-part-2/

from random import random
from math import floor

In [3]:
class Gene():
    
    #  This is the constructor method of the Gene class. 
    # It initializes a new instance of Gene with a code attribute and sets 
    # the cost attribute to 9999.
    def __init__(self, code):
        self.code = code
        self.cost = 9999

    # This method takes another Gene object as input and performs mating or 
    # crossover operation between the current gene and the input gene. It 
    # splits the code of both genes at the middle index and creates two new 
    # genes by combining the first half of the code from one gene with the 
    # second half of the code from the other gene, and vice versa. It returns 
    # a list containing the two new genes created.    
    def mate(self, gene):
        middle = int(floor(len(self.code)/2))
        return [Gene(self.code[:middle] + gene.code[middle:]),
                Gene(gene.code[:middle] + self.code[middle:])]

    #This method introduces random mutations in the code of the gene based 
    # on a given chance value. If a randomly generated number is less than 
    # chance, the method performs the mutation. It selects a random index in 
    # the code and either increments or decrements the ASCII value of the 
    # character at that index by 1, with a restriction that the new ASCII 
    # value should be within the range of 1 to 255.
    def mutate(self, chance):
        if random() < chance:
            return

        code = ''
        index = round(random() * len(self.code))
        for i in range(len(self.code)):
            upOrDown = -1 if round(random()) else 1
            if i == index and ord(self.code[i]) + upOrDown < 256 and ord(self.code[i]) > 0:
                code += chr(ord(self.code[i]) + upOrDown)
            else:
                code += self.code[i]

        self.code = code

    # This method generates a random code of a specified length by creating a 
    # string of characters with random ASCII values ranging from 0 to 255.
    def random(self, length):
        code = ''
        for i in range(length):
            code += chr(int(random()*255))
        self.code = code

    # This method calculates the cost of the gene by comparing its code with a 
    # given target string. It iterates over each character in the code and calculates 
    # the squared difference between the ASCII values of the character in the gene and 
    # the corresponding character in the target string. The sum of all squared differences 
    # is stored in the cost attribute of the gene.
    def calcCost(self, target):
        total = 0
        for i in range(len(self.code)):
            total += (ord(self.code[i]) - ord(target[i])) * (ord(self.code[i]) - ord(target[i]))

        self.cost = total


In summary, this code represents a genetic algorithm where a gene is represented by a string of characters. The gene can mate with another gene, mutate randomly, generate a random gene, and calculate its cost based on a target string. The genetic algorithm can be used for optimization problems where the aim is to find a gene that minimizes the cost.

In [4]:
class Population():
    
    # This is the constructor method of the Population class. It initializes a new 
    # instance of Population with the given target string, size (the number of members 
    # in the population), and log_costs (a boolean flag indicating whether to log the 
    # costs of the highest ranking member). It creates a list of Gene objects as the 
    # population members, where each gene has a random code of the same length as the 
    # target string. It also initializes the generationNumber attribute to 0.
    def __init__(self, target, size, log_costs):
        self.target = target
        self.members = []
        for i in range(size):
            gene = Gene('')
            gene.random(len(self.target))
            self.members.append(gene)
        self.generationNumber = 0
        
        self.log_costs = log_costs
        if self.log_costs:
            self.cost_log = [] # logs the cost of the highest ranking member
        
    # This method calculates the cost for each member in the population by calling 
    # the calcCost() method of each Gene object.
    def calcCosts(self):
        for member in self.members:
            member.calcCost(self.target)

    # This method applies the mutate() method to each member in the population, 
    # which introduces random mutations in their codes based on the given chance value.
    def mutate(self, chance):
        for member in self.members:
            member.mutate(chance)

    # This method sorts the population members based on their costs in ascending order.
    # Should be called after Population.calcCosts()
    def sort(self):
        self.members = sorted(self.members, key=lambda member: member.cost)

    # This method calculates the costs, sorts the population, and displays the generation 
    # number, the code of the member with the lowest cost (highest ranking member), and its cost.
    def display(self):
        self.calcCosts()
        self.sort()
        print("Generation", self.generationNumber, self.members[0].code, self.members[0].cost)

    # This method performs a generation of the genetic algorithm. It repeatedly calls 
    # the _generation() method until a member in the population has the same code as 
    # the target string. If display is True, it displays the population details during 
    # each generation. If log_costs is True, it returns a list containing the costs of 
    # the highest ranking member at each generation; otherwise, it returns the total 
    # number of generations.
    def generation(self, display):
        while not self._generation(display):
            pass
            
        if self.log_costs:
            return self.cost_log
        else:
            return self.generationNumber
        
    # This method performs a single generation of the genetic algorithm. It calculates costs, 
    # sorts the population, logs the cost if log_costs is True, displays the population if 
    # display is True, performs mating between the two highest ranking members, replaces the 
    # two lowest ranking members with the children generated from mating, mutates all members 
    # (except the highest ranking member), recalculates their costs, checks if any member has 
    # the same code as the target string, and updates the generation number.
    def _generation(self, display):
        self.calcCosts()
        self.sort()
        if self.log_costs:
            self.cost_log.append(self.members[0].cost)
        if display:
            self.display()

        children = self.members[0].mate(self.members[1])
        self.members[-2] = children[0]
        self.members[-1] = children[1]

        for member in self.members:
            member.mutate(0.5)
            member.calcCost(self.target)
            if member.code == self.target:
                self.sort()
                if display:
                    self.display()
                return True

        self.generationNumber += 1
        return False

if __name__ == '__main__':
    gene1 = Gene("AAAAAAAAAAAAAA")
    gene2 = Gene("AAAAAAAAAAAAAA")
    children = gene1.mate(gene2)
    for child in children:
        child.calcCost("Hi, mein Name ist Udo!")
        print(child.code, child.cost)

AAAAAAAAAAAAAA 16639
AAAAAAAAAAAAAA 16639


The code outside the class defines two genes with initial codes, performs mating between them, calculates the costs of the children, and prints their codes and costs.

In summary, this code represents a population of genes where each gene's code evolves over generations through mating and mutation. The genetic algorithm aims to find a gene with a code that matches the target string. The population evolves through generations, and the highest ranking member (lowest cost) is continually improved until it matches the target string.

In [5]:
population = Population("Hi, mein Name ist Udo!", size=100, log_costs=True)
population.generation(display=True)
print(population.cost_log)

Generation 0 b7ÎÓ3\"ÁÖª\{zk 102067
Generation 1 b7ÎÓ3\"NvXKÒi\ 100136
Generation 2 b7ÎÓ3\"NvXKÒi\ 100136
Generation 3 b7ÎÓ3\"NvXKÒi[ 100019
Generation 4 c9ÎÓ3\"NvXKÒi[ 99940
Generation 5 b8ÎÓ3\"NvXKÒi[ 99912
Generation 6 c9ÎÓ3\#NvXKÒi[ 99815
Generation 7 c9ÎÓ3\"NvXKÑi[ 99721
Generation 8 c9ÎÓ3\#OvXKÑi[ 99551
Generation 9 c9ÎÓ3\#OvXKÑj[ 99540
Generation 10 c:ÎÓ3\#OvXKÑj[ 99437
Generation 11 c:ÎÓ3\#OvXKÑi[ 99351
Generation 12 c:ÎÓ3\#OvXKÑi[ 99351
Generation 13 c:ÎÓ3\#OvXK	Ñi[ 99198
Generation 14 c:ÎÓ3\#OvXK	Ñi[ 99198
Generation 15 c:ÎÓ3\#OvXK	Ñi[ 99198
Generation 16 c:ÎÓ2\#OvXK	Ñi[ 99161
Generation 17 c:ÎÓ3\#OvXK	Ñi[ 99103
Generation 18 c:ÎÓ2\#OvXK	Ñi[ 99066
Generation 19 c:ÎÓ3\#OvXK	Ði[ 98981
Generation 20 c:ÎÒ3\#OvXK	Ñi[ 98902
Generation 21 c:ÎÒ3\#PuXLÑi[ 98747
Generation 22 c:ÎÒ3\#PuXLÑi[ 98747
Generation 23 c;ÎÒ3\#OvXK	Ði[ 98679
Ge

Generation 256 &^CÀÃ/Y,Pj\RyÇiS 76585
Generation 257 &^BÀÃ/Y,Pi\RyÇjS 76423
Generation 258 &^CÀÃ/Y,Pi\RyÇjS 76338
Generation 259 &^CÀÃ/Y,Pi\RyÇjS 76338
Generation 260 &^CÀÃ/Y,Ph\RyÇjS 76193
Generation 261 &^B¿Ã/Y,Ph\RyÇjS 76097
Generation 262 &^B¿Â/Y,Ph\RyÇjS 75928
Generation 263 &^B¿Â/Y,Ph\RyÇjS 75928
Generation 264 &^B¿Â/Y,Ph\RyÇjR 75829
Generation 265 &^B¿Â/Y,Ph\RyÇjR 75829
Generation 266 &]B¿Â/Y,Ph\RyÇjS 75805
Generation 267 &]B¿Â/Y,Ph]RyÇjS 75780
Generation 268 &]B¾Â/Y,Ph]RyÇjS 75601
Generation 269 &]B¿Â/Y,Ph]RyÇjS 75589
Generation 270 ']B¿Â/Y,Ph]RyÇjS 75456
Generation 271 ']B¿Â/Y,Ph]RyÇjS 75267
Generation 272 ']B¿Â/Y,Ph]RyÇjS 75267
Generation 273 ']B¿Â/Y,Ph]RyÇjS 75310
Generation 274 ']B½Â/Y,Ph]RyÇjS 75143
Generation 275 ']B½Â/Y,Ph]RyÇjS 75143
Generation 276 ']B½Â/Y,Ph^RyÇjS 75120
Generation 277 ']B¼Â/Y,Ph]RyÇjS 74968
Generation 278 (]B½Â/Y,Ph^R

Generation 547 1|RJ²·-W4Q[dVg³jK 53683
Generation 548 0|QJ²·-W4Q[dVg³jK 53652
Generation 549 1|RJ²¶-W4QZdVg´jK 53580
Generation 550 0|QJ²·-W4Q[dVf³jK 53511
Generation 551 0|RJ²¶-W4Q[dVf³jK 53510
Generation 552 0|RJ²¶-W5Q[dVf³jK 53421
Generation 553 0|RJ±¶-W5Q[dVf³jK 53233
Generation 554 0|RJ±¶-W5QZdVf³jJ 53068
Generation 555 0|RJ±¶-W5QZdWf³jJ 53011
Generation 556 0|RJ±¶-W5QZdWf³jJ 53011
Generation 557 0|RJ±¶-W5QZdWf³jJ 53011
Generation 558 0|RJ±¶-W5QZdWf³jI 52930
Generation 559 0|RJ±¶-W5QYdWf³jJ 52896
Generation 560 0|RJ±µ-W5QYdWf³jJ 52753
Generation 561 0{RJ±µ-W5QYdWf³jJ 52594
Generation 562 0{RJ±µ-W5QYdWf³jJ 52594
Generation 563 0{RJ±µ-W5QYdWf³jJ 52594
Generation 564 0{RJ°µ-W5QYdWf³jJ 52443
Generation 565 0{RJ°µ-W5QYdWf³jJ 52443
Generation 566 0{RJ±´-W5QYdWf²jJ 52296
Generation 567 0{RJ±´-W5QYdWf²jJ 52195
Generation 568 0{RJ±²-W5QYdWf²jJ 52020
Generation 569 0{RJ±²-W5RYdW

Generation 762 4rMP¥|«,V:TSeY\¥kD 39619
Generation 763 4rMP¥|«,V;TSeY\¤kD 39413
Generation 764 4rMP¥|«,V;TSeY\¤kD 39413
Generation 765 4rMP¥{«,V;TSeY[¤kD 39257
Generation 766 4rMP¥{«,V;TSeY[¤kD 39257
Generation 767 4rMP¤{«,V;TSeY[¤kD 39130
Generation 768 4rMP¤{«,V;TSeY[¤kD 39130
Generation 769 4rMP¤{«,V;TSeY[¤kD 39130
Generation 770 4rMP¤{«,V;TSeY[¤lD 39123
Generation 771 4rLP¤{«,V;TSdY[¤kD 39050
Generation 772 4rLP¤{«,V;TSeZ[¤kD 38990
Generation 773 4rLQ¤{«,V;TReZ[¤kD 38832
Generation 774 4rKQ¤{«,V;TReZ[¤kD 38745
Generation 775 4rKQ¤{«,V;TReZ[¤kD 38745
Generation 776 4rKQ¤{«+V;TReZ[¤kD 38722
Generation 777 4rKQ¤{«+V;TQeZ[¤kD 38623
Generation 778 4rKQ¤{«+V;TQeZ[¤kD 38623
Generation 779 5rKQ¤{«+V;TQeZ[¤kD 38518
Generation 780 5rKQ¤{«+V;TQe[[¤kD 38469
Generation 781 4rKQ¤{«+V;TQeZ[¤kD 38380
Generation 782 4rKQ¤{«+V;TQeZ[¤kD 38345
Generation 783 4rKQ¤{«+V;TQeZ[¤kD 38345
Generation 784 5rKQ¤{«+V<TQeZ

Generation 981  >hGUy¡'TDYIh\P&nB 27714
Generation 982 >hGUy¡'TDXHh\P&mB 27693
Generation 983  >hGUy¡'TDXHh\P&mB 27612
Generation 984  >hGUy'TCXIh\P&mB 27552
Generation 985  >hGUy'TCXHh\P&mB 27471
Generation 986  >hGUy'TCXIh\P&mB 27455
Generation 987  >hGUy'TCXHh\P&mB 27374
Generation 988  ?hGUy'TCXHh\P&mB 27289
Generation 989  ?hGUy'TCXHh\P&mB 27289
Generation 990  ?hGUy'TCXHh\P&mB 27289
Generation 991  >hGUy'TCXHi\P&mB 27268
Generation 992  >hGUy'TCXHh\O&nB 27171
Generation 993  >hGUy'TCXHh\N&nB 27078
Generation 994  ?gGUy(TCXHh\N&nB 26994
Generation 995  ?gGUy(TDXHh\N&nB 26935
Generation 996  ?gGUy(TCXHh\M&nB 26903
Generation 997  ?gGUy(TDXHh\M&nB 26844
Generation 998  ?gGUy(TDXGh\N&oB 26855
Generation 999  ?gFUy(TDXHh\N&nB 26835
Generation 1000  ?gFUy(TDXGh\N&oB 26778
Generation 1001  ?gGUy)TDXHh\L&nB 26772
Generation 1002  ?gFUy(TDXHh\L&oB 26677
Generation 1003 !?gFUy(TDX

Generation 1222 !GXA\y*SJ~ZBiczD0o> 17686
Generation 1223 !GXA\y*SJ~ZBiczC0o> 17615
Generation 1224 !GWA\y*RI~ZBiczC0o> 17566
Generation 1225 !GW@\y*RI~ZBiczC0o> 17501
Generation 1226 !GW@\y)RI~ZBiczC0o> 17482
Generation 1227 !GW@\y*QI~ZBiczC1o> 17421
Generation 1228 !GW@\y)RI~ZBiczC1p> 17410
Generation 1229 !GW@\y*QI~[BiczC1o> 17400
Generation 1230 !GW@\y)RI~[BiczC1o> 17388
Generation 1231 !GW@\y)RI~[BiczC1o> 17388
Generation 1232 !GW@\y)RI~[AiczC1o> 17248
Generation 1233 !GW@\y)RI~[AiczC1o> 17161
Generation 1234 !GW@\y)RI~[Aic{C1o> 17174
Generation 1235 !GW@\y)RI~[Aic{C1o> 17174
Generation 1236 !GW@\y)RI~[@ic{C1o> 17109
Generation 1237 !GW@\y)RI~[?ic{C1o> 17046
Generation 1238 !GW@\y)RI~[?id{C1o> 17015
Generation 1239 !GV@\y)SI~[@ic{C1o> 16962
Generation 1240 !GV@\y)SI~[?id{C1o> 16868
Generation 1241  GV@\y)SI~[?id{C1o> 16947
Generation 1242 !GV@\y*SI~[?id{C1o> 16887
Generation 1243 !GV@\y)SI~[?id{C1o> 16868
Generation

Generation 1456 &OK7`s)PN{]8jhx;<q7 10231
Generation 1457 &OK7`s)PN{]8jgx;<q6 10211
Generation 1458 &OK7`s)PN{]8jhx:<r7 10183
Generation 1459 &OJ7`s)PN{]8jhx:<r7 10122
Generation 1460 &OJ7`s(PN{]8jhx:<r7 10105
Generation 1461 &OK7`s)PN{]8jhx9<r7 10063
Generation 1462 &OK7`s)PNz]8jhx;<q6 10019
Generation 1463 &OK7`s)PNz]8jix;<q6 9998
Generation 1464 &OK7`s)PNz]8jhx;<q5 9978
Generation 1465 &PK7`s)POz]8jhx;<q5 9959
Generation 1466 &OK7`s)PNz]8jhx;<q5 9923
Generation 1467 &PK7`s)POz]8jhx;<q5 9890
Generation 1468 &OK6as)PNz]8jhx;<q5 9853
Generation 1469 &OK6as)PNz]8jhx;<q5 9853
Generation 1470 &OK7`s)PNz]8jhx;<r5 9808
Generation 1471 &OK7`s)PNz]8jix;=q5 9733
Generation 1472 &OK7`s)PNz]8jix;>q5 9686
Generation 1473 &OK7`s)PNz]8jix;>q5 9686
Generation 1474 &OK7`s)PNz]8jix;>q5 9686
Generation 1475 &OK7`s)PNz]8jix;>q5 9615
Generation 1476 &OK7`s)PNz]8jix;>q5 9615
Generation 1477 'OK7`s)PNz]8jix;>q5 9548
Generation 1478 'OK7`s)

Generation 1682 -[E-bq}'LRz^.ilx4Dzn+ 4889
Generation 1683 -\E-bq}'LRz^.ilx4Dzn+ 4862
Generation 1684 -\E-bq}'LRz^.ilx4Dzn+ 4862
Generation 1685 -\E-bq}'LRz^.ilx4Dzn+ 4862
Generation 1686 -\E-bq}'LSz^.ilx4Dzn+ 4833
Generation 1687 -\E-bq}'LSz^.ilx4Ezn+ 4800
Generation 1688 -[D-bq}'KSz^/ilx4Dzo+ 4791
Generation 1689 -[D-bq}'KSz^.ilx4Ezn+ 4730
Generation 1690 -[D-bq}'KSz^.ilx4Ezn+ 4730
Generation 1691 -[D-bq}'KSz^.ilx4Fzn+ 4699
Generation 1692 -[D-bq}'KSz^.ilx4Fzn+ 4699
Generation 1693 -[D,bq|'KSz^.hlx4Ezn+ 4677
Generation 1694 -[D,bq|'KSz^.ilx4Fzn+ 4645
Generation 1695 -[D,bq|'KSz^.ilx4Fzn+ 4645
Generation 1696 -[D,bq|'KSz_.ilx4Fzn+ 4632
Generation 1697 .[D-bo}(KTz_.ilx4Fzn+ 4593
Generation 1698 .[D-bo}(KTz_.ilx4Fzn+ 4593
Generation 1699 .[D-bo}(KTz_.ilx4Fyn+ 4550
Generation 1700 .[C-bo}(KTz_.ilx4Fzn+ 4546
Generation 1701 .[C-bn}(KTz_.ikx4Fyn+ 4507
Generation 1702 .[C-cn}(KTz_.ikx4Fyn+ 4486
Generation 1703 .[B-cn}(KTz_.ilx4Fzn* 4450
Generation 1704 .[B-cn}(KTz_.ik

Generation 1972 5a9$jkmr$LXpb+glv(Rlp& 1147
Generation 1973 5a9$jklr$LXpb+glv(Rlp& 1140
Generation 1974 5a9$jklr$LXpb*glv(Rlp& 1119
Generation 1975 5a9$jklr$LXpb*glv(Rlp& 1119
Generation 1976 5a9$jklr$LXpb+glv'Rlp& 1125
Generation 1977 5a9#jklr$LXpb+glv'Rlp& 1118
Generation 1978 5a9#jklr$LXpb*glv'Rlp& 1097
Generation 1979 5a9#jklr$LXpb*glu'Rlp& 1094
Generation 1980 5a9#jklr$LXpb*hlu'Rlp& 1091
Generation 1981 5a9#jklr$LXpb*hlu'Rlp& 1091
Generation 1982 5a9#jkkr$LXpb*hlu'Rlp& 1086
Generation 1983 5a8#jklr$LXpb+glv'Rkp& 1078
Generation 1984 5a8#jklr$LXpb*hlu'Rlp& 1066
Generation 1985 5a8#jklr$LXpb*hlu'Rlp& 1066
Generation 1986 5a8#jkkr$LXpb*hlu'Rlp& 1061
Generation 1987 5a7#jklr$LXpb*hku'Rlp& 1058
Generation 1988 5a7#jkkr$LXpb*hlu'Rlp& 1038
Generation 1989 5a7#jkkr$LXpb*ilu'Rlp& 1037
Generation 1990 5a7#jkkr$LXpb*ilu'Rlp& 1037
Generation 1991 5a7#jkkr$LXob*ilu'Rlp& 1032
Generation 1992 5a7#jkkr$LXob*imu'Rlp& 1019
Generation 1993 5a7#jkkr$KXpb*imu'Rlo& 1028
Generation 1994 5a7"jkkr$LXpb*im

Generation 2228 Fh. mein N`mf!ist!Sdo! 17
Generation 2229 Fh. mein N`mf!ist!Sdo! 17
Generation 2230 Fh. mein N`mf!ist!Sdo! 17
Generation 2231 Fh. mein N`mf!ist!Sdo! 17
Generation 2232 Fh. mein N`mf!ist Sdo! 16
Generation 2233 Fi. mein N`mf!ist!Sdo! 16
Generation 2234 Gh. mein N`mf!ist!Sdo! 14
Generation 2235 Gh. mein N`mf!ist!Tdo! 11
Generation 2236 Gh- mein N`mf!ist!Sdo! 11
Generation 2237 Gh- mein N`mf!hst!Tdo  10
Generation 2238 Gh- mein N`mf!hst!Tdo  10
Generation 2239 Gh- mein N`mf!hst!Tdo  10
Generation 2240 Gh- mein N`mf!hst!Tdo  10
Generation 2241 Gh- mein N`mf!hst!Tdo  10
Generation 2242 Gh- mein N`mf!hst!Udo  9
Generation 2243 Gh- mein N`me!hst!Udo  8
Generation 2244 Gh- mein N`me!hst!Udo  8
Generation 2245 Gh- mein N`me!hst!Udo  8
Generation 2246 Gh- mein N`me!ist!Udo  7
Generation 2247 Gh, mein N`me hst!Udo  6
Generation 2248 Gh, mein N`me hst!Udo  6
Generation 2249 Gi, mein N`me hst!Udo  5
Generation 2250 Gi, mein N`me hst!Udo  5
Generation 2251 Gi, mein N`me!ist!Udo  5
Ge